In [2]:
import requests

In [3]:
standings_url = "https://www.sports-reference.com/cfb/schools"

In [4]:
data = requests.get(standings_url)

In [5]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(data.text)

In [7]:
standings_table = soup.select('table.stats_table')[0]

In [8]:
links = standings_table.find_all('a')

In [9]:
links = [l.get("href") for l in links]

In [10]:
links = [l for l in links if 'schools' in l]

In [11]:
team_urls = [f"https://sports-reference.com{l}" for l in links]

In [12]:
schedule = ['2021-schedule.html', '2022-schedule.html', '2023-schedule.html']

gamelog = ['2021/gamelog/', '2022/gamelog/', '2023/gamelog/']

In [13]:
schedule_urls = [i + schedule[0] for i in team_urls]

In [14]:
schedule_url = schedule_urls

In [15]:
len(schedule_url)

301

In [16]:
data = requests.get(schedule_url[0])

In [17]:
import pandas as pd

games = pd.read_html(data.text, match="Games")

C:\Users\parke\AppData\Local\Temp\ipykernel_9112\2224478326.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  games = pd.read_html(data.text, match="Games")


In [18]:
games[0].rename(columns = {'Unnamed: 5':'Location'}, inplace = True)
games[0].rename(columns = {'Unnamed: 8':'Outcome'}, inplace = True)
games[0]['Location'] = games[0]['Location'].fillna('H')
games[0] = pd.concat([games[0]], keys=['Game Info'], axis=1)
games[0].head()

Game Info                                                                    \
          G          Date     Time  Day     School Location          Opponent   
0         1   Sep 4, 2021  2:00 PM  Sat  Air Force        H         Lafayette   
1         2  Sep 11, 2021  3:30 PM  Sat  Air Force        @              Navy   
2         3  Sep 18, 2021  7:30 PM  Sat  Air Force        H        Utah State   
3         4  Sep 25, 2021  8:00 PM  Sat  Air Force        H  Florida Atlantic   
4         5   Oct 2, 2021  6:30 PM  Sat  Air Force        @        New Mexico   

                                                 
        Conf Outcome Pts Opp  W  L Streak Notes  
0  Non-Major       W  35  14  1  0    W 1   NaN  
1   American       W  23   3  2  0    W 2   NaN  
2        MWC       L  45  49  2  1    L 1   NaN  
3       CUSA       W  31   7  3  1    W 1   NaN  
4        MWC       W  38  10  4  1    W 2   NaN

In [19]:
gamelogs = [i + gamelog[0] for i in team_urls]

In [20]:
gamelogs[0]

'https://sports-reference.com/cfb/schools/air-force/2021/gamelog/'

In [21]:
data = requests.get(gamelogs[0]).text.replace('<!--', '').replace('-->', '')

In [22]:
offGameLog = pd.read_html(data, match="Offensive Game Log Table")

C:\Users\parke\AppData\Local\Temp\ipykernel_9112\635328836.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  offGameLog = pd.read_html(data, match="Offensive Game Log Table")


In [23]:
defGameLog = pd.read_html(data, match="Defensive Game Log")

C:\Users\parke\AppData\Local\Temp\ipykernel_9112\275867740.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defGameLog = pd.read_html(data, match="Defensive Game Log")


In [24]:
columns_to_keep = ["Passing", "Rushing", "Total Offense", "Turnovers"]

# Drop all other columns
offGameLog[0] = offGameLog[0][columns_to_keep]

In [25]:
new_columns = pd.MultiIndex.from_tuples([
    ('Off Passing', 'Cmp'),
    ('Off Passing', 'Att'),
    ('Off Passing', 'Pct'),
    ('Off Passing', 'Yds'),
    ('Off Passing', 'TD'),
    ('Off Rushing', 'Att'),
    ('Off Rushing', 'Yds'),
    ('Off Rushing', 'Avg'),
    ('Off Rushing', 'TD'),
    ('Off Total Offense', 'Plays'),
    ('Off Total Offense', 'Yds'),
    ('Off Total Offense', 'Avg'),
    ('Off Turnovers', 'Fum'),
    ('Off Turnovers', 'Int'),
    ('Off Turnovers', 'TO')
])

# Assign the new column structure to the DataFrame
offGameLog[0].columns = new_columns

offGameLog[0].head()

Off Passing                   Off Rushing              Off Total Offense  \
          Cmp Att   Pct  Yds TD         Att  Yds  Avg TD             Plays   
0           3   5  60.0   50  0          67  370  5.5  5                72   
1           3  10  30.0   49  0          59  176  3.0  3                69   
2           6  12  50.0  182  1          61  438  7.2  5                73   
3           2   5  40.0   70  0          73  446  6.1  4                78   
4           1   2  50.0   33  0          73  408  5.6  5                75   

            Off Turnovers         
   Yds  Avg           Fum Int TO  
0  420  5.8             0   0  0  
1  225  3.3             0   0  0  
2  620  8.5             1   1  2  
3  516  6.6             0   0  0  
4  441  5.9             0   0  0

In [26]:
columns_to_keep = ["Passing", "Rushing", "Total Offense", "Unnamed: 23_level_0", "Unnamed: 24_level_0", "Unnamed: 25_level_0"]

# Drop all other columns
defGameLog[0] = defGameLog[0][columns_to_keep]

In [27]:
new_columns = pd.MultiIndex.from_tuples([
    ('Def Passing', 'Cmp'),
    ('Def Passing', 'Att'),
    ('Def Passing', 'Pct'),
    ('Def Passing', 'Yds'),
    ('Def Passing', 'TD'),
    ('Def Rushing', 'Att'),
    ('Def Rushing', 'Yds'),
    ('Def Rushing', 'Avg'),
    ('Def Rushing', 'TD'),
    ('Def Total Offense', 'Plays'),
    ('Def Total Offense', 'Yds'),
    ('Def Total Offense', 'Avg'),
    ('Def Turnovers', 'Fum'),
    ('Def Turnovers', 'Int'),
    ('Def Turnovers', 'TO')
])

# Assign the new column structure to the DataFrame
defGameLog[0].columns = new_columns

In [28]:
defGameLog[0].head()

Def Passing                   Def Rushing              Def Total Offense  \
          Cmp Att   Pct  Yds TD         Att  Yds  Avg TD             Plays   
0          18  29  62.1  243  1          23   44  1.9  1                52   
1           3  10  30.0   32  0          34   36  1.1  0                44   
2          31  50  62.0  448  5          39  180  4.6  2                89   
3          11  33  33.3   78  0          20  141  7.1  1                53   
4          14  23  60.9  179  1          20   47  2.4  0                43   

            Def Turnovers         
   Yds  Avg           Fum Int TO  
0  287  5.5             0   1  1  
1   68  1.5             1   0  1  
2  628  7.1             0   1  1  
3  219  4.1             0   1  1  
4  226  5.3             2   1  3

In [29]:
team_data = offGameLog[0].merge(games[0], left_index=True, right_index=True)
team_data = defGameLog[0].merge(games[0], left_index=True, right_index=True)
team_data.head()

Def Passing                   Def Rushing              Def Total Offense  \
          Cmp Att   Pct  Yds TD         Att  Yds  Avg TD             Plays   
0          18  29  62.1  243  1          23   44  1.9  1                52   
1           3  10  30.0   32  0          34   36  1.1  0                44   
2          31  50  62.0  448  5          39  180  4.6  2                89   
3          11  33  33.3   78  0          20  141  7.1  1                53   
4          14  23  60.9  179  1          20   47  2.4  0                43   

   ... Game Info                                                            \
   ...  Location          Opponent       Conf Outcome Pts Opp  W  L Streak   
0  ...         H         Lafayette  Non-Major       W  35  14  1  0    W 1   
1  ...         @              Navy   American       W  23   3  2  0    W 2   
2  ...         H        Utah State        MWC       L  45  49  2  1    L 1   
3  ...         H  Florida Atlantic       CUSA       W  31   7  3  1    W 1   
4  ...         @        New Mexico        MWC       W  38  10  4  1    W 2   

         
  Notes  
0   NaN  
1   NaN  
2   NaN  
3   NaN  
4   NaN  

[5 rows x 30 columns]

In [30]:
years = list(range(2022,2020,-1))

In [31]:
all_games = 

SyntaxError: invalid syntax (886569297.py, line 1)

In [35]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if 'schools' in l]
    team_urls = [f"https://sports-reference.com{l}" for l in links]

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-", " ")
        data = requests.get(schedule_url))
        games = pd.read_html(data.text, match="Games")



        team_data["Season"] = year
        team_data["Team] = team_name
        all_games.append(team_data)
        time.sleep(1)

SyntaxError: unmatched ')' (2482734351.py, line 11)

In [34]:
team_url

NameError: name 'team_url' is not defined

In [ ]:
team_url.split("/")[-2].replace("-", " ").capwords()